In [1]:
import random
from Utility import df_ops, music_ops, vmd_ops, io_ops, interpolate, cloud_ops
import pandas as pd
import os 
import pinecone
import pymeshio.common
import numpy as np

MIN_FRAMES = 30
MAX_FRAMES = 60
PAST_SONG_REPEAT_DISTANCE = 3
FILE_NAME = "shortNsweet"
BUFFER = 4

class DanceClip:
  def __init__(self, pathToVmdFolder, startFrame, numberOfFrames):
    self.pathToVmdFolder = pathToVmdFolder
    self.startFrame = startFrame
    self.numberOfFrames = numberOfFrames
  
  def __str__(self):
    return "DanceClip: " + self.pathToVmdFolder + " startFrame: " + str(self.startFrame) + " numberOfFrames: " + str(self.numberOfFrames)

def queryForSimilarPoses(pinecone, vector, songNamesToExclude):
  index = pinecone.Index(os.getenv('PINECONE_INDEX_NAME'))
  songs = index.query(
    vector=vector,
    filter={
      "animationName": { "$nin": songNamesToExclude[-PAST_SONG_REPEAT_DISTANCE:] }
    },
    top_k=1,
    include_metadata=True
  )
  matchedSongName = songs.matches[0].metadata['animationName']
  frameNumberOfMatchedSong = songs.matches[0].metadata['frameNumber']
  distance = songs.matches[0]['score']
  print("found match: " + matchedSongName + " at frame: " + str(frameNumberOfMatchedSong) + " with distance: " + str(distance))
  return matchedSongName, frameNumberOfMatchedSong

def appendFramesPreservingCenter(df, dfToAppend, buffer=0):
  if df.empty:
    lastPositionOfDf = pymeshio.common.Vector3(0, 0, 0)
  else:
    lastPositionOfDfIdx = df[df['name'] == 'センター']['frame'].idxmax()
    lastPositionOfDf = df.iloc[lastPositionOfDfIdx]['position']

  try:
    firstPositionOfDfToAppendIdx = dfToAppend[dfToAppend['name'] == 'センター']['frame'].idxmin()
    firstPositionOfDfToAppend = dfToAppend.iloc[firstPositionOfDfToAppendIdx]['position']
  except:
    if dfToAppend.empty:
      print("dfToAppend is empty")
    if len(dfToAppend[dfToAppend['name'] == 'センター']) == 0:
      print("No rows with name 'センター' in dfToAppend")
    print("Index to access:", firstPositionOfDfToAppendIdx)
    print("Length of dfToAppend:", len(dfToAppend))
    raise Exception("Error")

  differenceInPos = firstPositionOfDfToAppend - lastPositionOfDf
  differenceInPos.y = 0

  positionBoneNames = ['センター', '右足ＩＫ', '左足ＩＫ']
  dfToAppendWithAdjustedPositions = dfToAppend.copy()
  dfToAppendWithAdjustedPositions['position'] = np.where(dfToAppendWithAdjustedPositions['name'].isin(positionBoneNames), 
                                  dfToAppendWithAdjustedPositions['position'] - differenceInPos,
                                  dfToAppendWithAdjustedPositions['position'])
  return df_ops.appendFrames(df, dfToAppendWithAdjustedPositions, buffer=buffer)

def getVectorFromVectorId(pinecone, songName, framenumber):
  vectorId = songName + "-" + str(int(framenumber))
  index = pinecone.GRPCIndex(os.getenv('PINECONE_INDEX_NAME'))
  fetchVectorId = index.fetch([vectorId])
  return fetchVectorId['vectors'][vectorId]['values']

def generateAnimationClips(numberOfFrames):
  INPUT_FOLDER = 'inputData'
  pinecone = cloud_ops.initPinecone()
  sum = 0
  end = numberOfFrames
  newSongName = random.choice([entry for entry in os.listdir(INPUT_FOLDER) if os.path.isdir(os.path.join(INPUT_FOLDER, entry))])
  print("starting with song: " + newSongName)
  pastSongs = [newSongName]
  animationClips = []
  frameNumberOfMatchedSong = None

  while sum < end:
    if sum != 0:
      print("querying for similar poses for song: " + prevSongName + " at frame: " + str(startingFrameOfNewFrames + numberOfNewFramesToAdd))
      poseVector = getVectorFromVectorId(pinecone, prevSongName, startingFrameOfNewFrames + numberOfNewFramesToAdd)
      newSongName, frameNumberOfMatchedSong = queryForSimilarPoses(pinecone, poseVector, pastSongs)
      pastSongs.append(newSongName)

    vmd = [f for f in os.listdir(INPUT_FOLDER + "\\" + newSongName) if f.endswith('.vmd')][-1]
    df = vmd_ops.getDfFromVmdFileName(INPUT_FOLDER + "\\" + newSongName + "\\" + vmd)
    numberOfNewFramesToAdd = random.randint(MIN_FRAMES, MAX_FRAMES)
    if frameNumberOfMatchedSong and frameNumberOfMatchedSong + numberOfNewFramesToAdd > df_ops.getLastFrame(df): #handle if not enough frames left in song
      numberOfNewFramesToAdd = df_ops.getLastFrame(df) - frameNumberOfMatchedSong
    startingFrameOfNewFrames = frameNumberOfMatchedSong if sum != 0 else random.randint(0, df_ops.getLastFrame(df) - numberOfNewFramesToAdd)

    animationClips.append(DanceClip(INPUT_FOLDER + "\\" + newSongName, startingFrameOfNewFrames, numberOfNewFramesToAdd))
    sum += numberOfNewFramesToAdd
    prevSongName = newSongName
  
  return animationClips

def generateVmdAnimationFromDance(animationClips, savePath, saveName, bufferFrames=0, viewInterpolation=False):
  newAnimation = pd.DataFrame()
  newInterpolatedAnimation = pd.DataFrame()
  frameNumber = 0

  for animationClip in animationClips:
    vmd = [f for f in os.listdir(animationClip.pathToVmdFolder) if f.endswith('.vmd')][-1]
    df = vmd_ops.getDfFromVmdFileName(animationClip.pathToVmdFolder + "\\" + vmd)
    newAnimation = appendFramesPreservingCenter(newAnimation, df_ops.parseFramesFromDf(df, animationClip.startFrame, animationClip.startFrame + animationClip.numberOfFrames), bufferFrames)
    if viewInterpolation:
      interpolatedDf = df_ops.loadDfFromFeather(animationClip.pathToVmdFolder + "\\interpolatedMotion.feather")
      newInterpolatedAnimation = df_ops.appendFrames(newInterpolatedAnimation, df_ops.parseFramesFromDf(interpolatedDf, animationClip.startFrame, animationClip.startFrame + animationClip.numberOfFrames), bufferFrames)
    print("adding song: " + animationClip.pathToVmdFolder.split("\\")[-1] + "-" + str(animationClip.startFrame) + " at frame: " + str(frameNumber))
    frameNumber += animationClip.numberOfFrames + bufferFrames

  vmd_ops.saveDfToVmdFile(newAnimation, savePath + "\\" + saveName + ".vmd")
  if viewInterpolation:
    vmd_ops.saveDfToVmdFile(newInterpolatedAnimation, savePath + "\\" + saveName + "-interpolated-DEBUG.vmd")

  return newAnimation

animationClips = generateAnimationClips(1500)
animationDf = generateVmdAnimationFromDance(animationClips, 'outputMotions', FILE_NAME, bufferFrames=BUFFER, viewInterpolation=True)

c:\Users\daryl\AppData\Local\Programs\Python\Python311\Lib\site-packages\pinecone\index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


starting with song: PV027 - Ievan Polkka
querying for similar poses for song: PV027 - Ievan Polkka at frame: 347
found match: PV224 - Rosary Pale at frame: 4458.0 with distance: 0.991079807
querying for similar poses for song: PV224 - Rosary Pale at frame: 4507.0
found match: PV902 - Aidee at frame: 7869.0 with distance: 0.986007869
querying for similar poses for song: PV902 - Aidee at frame: 7904.0
found match: PV914 - Calc at frame: 6740.0 with distance: 0.983604
querying for similar poses for song: PV914 - Calc at frame: 6776.0
found match: PV931 - Sharing The World at frame: 884.0 with distance: 0.983595848
querying for similar poses for song: PV931 - Sharing The World at frame: 928.0
found match: PV043 - Romeo and Cinderella at frame: 1855.0 with distance: 0.983545482
querying for similar poses for song: PV043 - Romeo and Cinderella at frame: 1909.0
found match: PV913 - Even a Kunoichi Needs Love at frame: 2977.0 with distance: 0.975204587
querying for similar poses for song: PV91

In [2]:
for animationClip in animationClips:
  print(animationClip)

DanceClip: inputData\PV027 - Ievan Polkka startFrame: 311 numberOfFrames: 36
DanceClip: inputData\PV224 - Rosary Pale startFrame: 4458.0 numberOfFrames: 49
DanceClip: inputData\PV902 - Aidee startFrame: 7869.0 numberOfFrames: 35
DanceClip: inputData\PV914 - Calc startFrame: 6740.0 numberOfFrames: 36
DanceClip: inputData\PV931 - Sharing The World startFrame: 884.0 numberOfFrames: 44
DanceClip: inputData\PV043 - Romeo and Cinderella startFrame: 1855.0 numberOfFrames: 54
DanceClip: inputData\PV913 - Even a Kunoichi Needs Love startFrame: 2977.0 numberOfFrames: 60
DanceClip: inputData\PV096 - Sekiranun Graffiti startFrame: 2638.0 numberOfFrames: 38
DanceClip: inputData\PV255 - Skeleton Orchestra and Lilia startFrame: 3679.0 numberOfFrames: 42
DanceClip: inputData\PV007 - Strobo Nights startFrame: 2017.0 numberOfFrames: 47
DanceClip: inputData\PV626 - Weekender Girl startFrame: 5755.0 numberOfFrames: 59
DanceClip: inputData\PV212 - Colorful x Sexy startFrame: 1109.0 numberOfFrames: 60
Dance

In [5]:
import os
import wave
from scipy.signal import resample

def create_empty_wav(output_file):
  with wave.open(output_file, 'wb') as out_wav:
    out_wav.setparams((1, 2, 44100, 0, 'NONE', 'not compressed'))
    out_wav.writeframes(b'')

def create_empty_wav_with_params(input_file, output_file):
  with wave.open(input_file, 'rb') as wav:
    params = wav.getparams()

  with wave.open(output_file, 'wb') as out_wav:
    out_wav.setparams(params)
    out_wav.writeframes(b'')

def append_wav_files(input_file1, input_file2, output_file, start_second, end_second):
  with wave.open(input_file1, 'rb') as wav1, wave.open(input_file2, 'rb') as wav2:
    params1 = wav1.getparams()
    params2 = wav2.getparams()

    frames1 = wav1.readframes(wav1.getnframes())
    frames1 = np.frombuffer(frames1, dtype=np.int16)

    # Calculate the start and end frames for the segment in file2
    frame_rate1 = wav1.getframerate()
    frame_rate2 = wav2.getframerate()
    start_frame = int(start_second * frame_rate2)
    end_frame = int(end_second * frame_rate2)
    n_frames = end_frame - start_frame

    # Position the file pointer and read the segment frames
    wav2.setpos(start_frame)
    frames2 = wav2.readframes(n_frames)
    frames2 = np.frombuffer(frames2, dtype=np.int16)

    # Resample frames2 to match the frame rate of frames1
    resampled_frames2 = resample(frames2, int(len(frames2) * frame_rate1 / frame_rate2)).astype(np.int16)

    # Combine frames1 and resampled_frames2
    combined_frames = np.concatenate((frames1, resampled_frames2))

    with wave.open(output_file, 'wb') as out_wav:
      out_wav.setparams(params1)
      out_wav.writeframes(combined_frames.tobytes())

import wave
import array

def append_silence(input_file, output_file, silence_duration):
  with wave.open(input_file, 'rb') as wav:
    params = wav.getparams()
    frames = wav.readframes(wav.getnframes())

    # Create 1 second of silence
    frame_rate = params.framerate
    n_channels = params.nchannels
    sampwidth = params.sampwidth

    n_frames_silence = int(frame_rate * silence_duration)
    silence_frames = array.array("h", [0] * n_frames_silence * n_channels)

    # Convert the array to bytes
    silence_frames = silence_frames.tobytes()

    with wave.open(output_file, 'wb') as out_wav:
      out_wav.setparams(params)
      out_wav.writeframes(frames + silence_frames)

def get_wav_duration(wav_file_path):
  with wave.open(wav_file_path, 'rb') as wav_file:
    n_frames = wav_file.getnframes()
    frame_rate = wav_file.getframerate()
    duration = n_frames / float(frame_rate)
  return duration

def pullAudioFromAnimationClips(animationClips, savePath, saveName, buffer=0):
  totalFrames = 6
  newWaveName = savePath + "\\" + saveName + ".wav"
  createOutputFile = True
  for animationClip in animationClips:
    songWavName = [f for f in os.listdir(animationClip.pathToVmdFolder) if f.endswith('.wav')][-1]
    songWav = animationClip.pathToVmdFolder + "\\" + songWavName
    print(songWav)
    if createOutputFile:
      create_empty_wav_with_params(songWav, newWaveName)
      append_silence(newWaveName, newWaveName, 6 / 30) # initial silence
      createOutputFile = False
    startSecond = animationClip.startFrame / 30
    endSecond = (animationClip.startFrame + animationClip.numberOfFrames) / 30
    append_wav_files(newWaveName, songWav, newWaveName, startSecond, endSecond)
    totalFrames += animationClip.numberOfFrames
    print("added: " + str(endSecond - startSecond) + " seconds for " + str(animationClip.numberOfFrames) + " frames. Total frames now at: " + str(totalFrames) + " frames. Total calculated seconds: " + str(totalFrames / 30) + ". Total actual seconds now at: " + str(get_wav_duration(newWaveName)) + " seconds.")
    append_silence(newWaveName, newWaveName, buffer / 30)
    totalFrames += buffer

pullAudioFromAnimationClips(animationClips, 'outputMotions', FILE_NAME, buffer=BUFFER)

inputData\PV027 - Ievan Polkka\pv_027.wav
 calced DIFFERENCE:-1.1999999999999993
TOTAL FRAMES: 123480 frameRate: 44100
added: 1.1999999999999993 seconds for 36 frames. Total frames now at: 42 frames. Total calculated seconds: 1.4. Total actual seconds now at: 1.4 seconds.
inputData\PV224 - Rosary Pale\pv_224.wav
 calced DIFFERENCE:-1.6333333333333258
TOTAL FRAMES: 279298 frameRate: 44100
added: 1.6333333333333258 seconds for 49 frames. Total frames now at: 95 frames. Total calculated seconds: 3.1666666666666665. Total actual seconds now at: 3.166643990929705 seconds.
inputData\PV902 - Aidee\pv_902.wav
 calced DIFFERENCE:-1.1666666666666288
TOTAL FRAMES: 393956 frameRate: 44100
added: 1.1666666666666288 seconds for 35 frames. Total frames now at: 134 frames. Total calculated seconds: 4.466666666666667. Total actual seconds now at: 4.466621315192744 seconds.
inputData\PV914 - Calc\pv_914.wav
 calced DIFFERENCE:-1.200000000000017
TOTAL FRAMES: 511556 frameRate: 44100
added: 1.200000000000

In [ ]:
# multiply x and y quaternion by -1 and replace corresponding bone to flip aniamtion